# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 4. Microsoft GraphRAG Query构建流程详解</center>

&emsp;&emsp;在完成了`Microsoft GraphRAG`的索引构建及自定义接入图数据库`Neo4j`构建完整的知识图谱后，我们在上一小节课程中已经初步实践了可以通过`Cypher`语句来查询结构化数据中的信息。当然，传统的`Cypher`查询方式，需要我们具备一定的图数据库知识，对非技术人员来说，使用门槛较高。 而`Microsoft GraphRAG`则提供了一种更为直观、易用的查询方式，我们只需要输入自然语言查询，即可获得结构化的查询结果。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503041644948.png" width=80%></div>

&emsp;&emsp;这就需要我们了解`Microsoft GraphRAG`使用的第二阶段，即查询（Query）阶段。

&emsp;&emsp;索引阶段我们利用大语言模型结合提示工程，从非结构化文本（`.txt`、`.csv`）中提取出实体（Entities）与关系（Relationships），构建出了基础的`Knowledge Graph`，并且通过建立层次化的`community` 结构，`community` 以及`community_report` 的丰富语义，相较于传统基于`Cypher`的查询方式可以提供更多灵活性的`Query`操作，`Microsoft GraphRAG` 在项目开源之初是提供了`local` 和`global` 两种查询方式，分别对应了`local search` 和`global search`，而后在不断的迭代更新过程中，除了优化了`local search` 和`global search` 的效果，还新增了`DRIFT Search` 和`Multi Index Search` 作为扩展优化的可选项，以进一步丰富`Query`操作的多样性。

&emsp;&emsp;如下图所以，原图来源于`Microsoft GraphRAG` 的官方论文：https://arxiv.org/pdf/2404.16130

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131632035.png" width=80%></div>

&emsp;&emsp;`Microsoft GraphRAG` 在查询阶段构建的流程，相较于构建索引阶段会更为直观，核心的具体步骤包括：

1. 接收用户的查询请求。
2. 根据查询所需的详细程度，选择合适的社区级别进行分析。
3. 在选定的社区级别进行信息检索。
4. 依据社区摘要生成初步的响应。
5. 将多个相关社区的初步响应进行整合，形成一个全面的最终答案。

&emsp;&emsp;通过学习`Microsoft GraphRAG`索引构建的源码大家应该已经能够清晰的知道，`Indexing` 过程中并不是在创建完第一层社区后就停止了，而是是分层的。也就是说，当创建第一层社区（即基础社区）后，会将这些社区视为节点，进一步构建更高层级的社区。这种方法就实现在知识图谱中可以以不同的粒度级别上组织和表示数据。比如第一层社区可以包含具体的实体或数据，而更高层级的社区则可以聚合这些基础社区，形成更广泛的概览。

&emsp;&emsp;因此最核心的`Local Search` 和`Global Search` 的实现，就是源于不同的粒度级别而构建出来用于处理不同类型问题的`Pipeline`, 其中：

- `Local Search` 是基于实体的检索。
- `Global Search` 则是基于社区的检索。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503131302122.png" width=80%></div>

&emsp;&emsp;因此接下来，我们就分别从源码层面，来详细介绍`Local Search` 和`Global Search` 的实现原理，并实际操作不同检索方式的查询操作。

&emsp;&emsp;首先来介绍`Local Search`， 即本地检索。

# 1. Local Search 实现原理与源码解析

&emsp;&emsp;<font color=red> `Local Search` 即本地检索，是基于实体的检索。</font>本地搜索从相关实体开始，使用知识图谱来查找最相关的信息。例如，给定查询中的实体，使用的是连接节点的信息，通过辨识与查询相关的实体与关系，检索特定文本片段、摘要和关联性资料。


&emsp;&emsp;所以`Local Search` 本质上是基于实体的推理。特别适合回答“who”、“what”、“when” 类型的问题。

&emsp;&emsp;在`Microsoft GraphRAG` 源码中实现的内部原理如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141026335.png" width=80%></div>

&emsp;&emsp;接下来，我们就进入源码，逐层的解析`Local Search` 的实现过程。

## 1.1 构建索引

&emsp;&emsp;首先，我们还是在当前的运行环境下先构建索引。（如果已经构建过索引，则可以跳过此步骤）这里我们使用的还是单`txt`文件即`technology_companies.txt` 文件，切分的 `chunk_size` 为`300`，`chunk_overlap` 为`100`。在源码环境下，执行构建索引的命令为：

```bash
    poetry run poe index --root ./
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141034327.png" width=80%></div>

&emsp;&emsp;构建索引完成后，在`output`目录下依次生成`document.parent`、`text_units.parent`、`entities.parent`、`relationships.parent`、`communities.parent`以及`community_report.parent` 主要的索引，在检索过程中，会加载这些索引文件中的相关数据内容。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141040359.png" width=60%></div>

## 1.2 Local Search 源码执行方法

&emsp;&emsp;与`grahrang index` 命令类似，`Microsoft GraphRAG` 的检索工作流可以通过`graphrag query` 命令支持，在源码环境下其命令格式如何使用，我们可以通过`graphrag query --help` 命令来查看，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141043353.png" width=80%></div>

&emsp;&emsp;各个字段参数的含义如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>graphrag query 命令参数说明</font></p>
<div class="center">

| 参数名称                          | 类型    | 描述                                                                                     | 默认值         | 是否必需         |
|-----------------------------------|---------|------------------------------------------------------------------------------------------|----------------|------------------|
| `--method`                        | Tpye| 可以选择local、global、drift或basic算法。                                                                          | None           | 是               |
| `--query`                         | TEXT    | 要执行的查询，即提出的问题。                                                                          | None           | 是               |
| `--config`                        | PATH    | 要使用的配置文件路径。                                                                  | None           | 否               |
| `--data`                          | PATH    | 索引管道输出目录（即包含 parquet 文件的目录）。                                         | None           | 否               |
| `--root`                          | PATH    | 项目根目录的路径。                                                                      | .              | 否               |
| `--community-level`               | INTEGER | 从中加载社区报告的 Leiden 社区层级。较高的值表示来自较小社区的报告。                     | 2              | 否               |
| `--dynamic-community-selection`    |         | 使用动态社区选择的全局搜索。                                                            | no-dynamic-community-selection | 否               |
| `--response-type`                 | TEXT    | 描述响应类型和格式的自由文本，可以是任何内容，例如多个段落、单个段落、单句、3-7点列表、单页、多页报告。 | Multiple Paragraphs | 否               |
| `--streaming`                     |         | 以流式方式打印响应。                                                                    | no-streaming    | 否               |
| `--help`                          |         | 显示帮助信息并退出。                                                                    |                | 否               |

</div>

&emsp;&emsp;其中，在执行查询时必须指定的参数是`--method`和`--query`，其他参数为可选参数。其中：

- `--method` 参数可以选择`local`、`global`、`drift`或`basic`算法。(接下来我们会依次介绍这几种算法)
- `--query` 参数是要执行的查询，即提出的问题。

&emsp;&emsp;了解到这里，就可以通过命令行快速启动问答检索了。这里我们先来看`local` 本地搜索。输入如下命令：

```bash
    poetry run poe query --root ./ --method local --query "苹果公司都有哪些产品？"
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141054125.png" width=80%></div>

&emsp;&emsp;最终会显示`SUCCESS: Local Search Response` 成功提示，并会显示最终的问答结果。整个过程使用起来非常简单，但是简单并不意味着可以直接使用，大多数情况下基于通用流程的问答检索，并不能满足实际业务需求。比如检索的效果不准确，效率不高，检索结果不全面等，因此，我们需要进一步掌握`Microsoft GraphRAG` 的检索原理，并根据实际业务需求，进行针对性的优化和调整。

&emsp;&emsp;所以接下来，我们就深入源码底层，详细的给大家介绍一下 `Local Search` 的完整实现过程，并对其中的一些优化策略进行详细的介绍。

## 1.3 Local Search 源码底层解析

&emsp;&emsp;`Microsoft GraphRAG` 查询阶段的入口文件与`Indexing` 阶段一样，启动的主入口文件仍然在`graphrag/cli/main.py` 文件中，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141105060.png" width=80%></div>

&emsp;&emsp;我们首先来看`local` 本地搜索的实现。在`main.py` 文件中，会根据`--method` 参数的值，来决定执行哪种检索方式。当指定了`local` 本地搜索时，会执行`run_local_search` 函数。（同理，`global` 全局搜索时，会执行`run_global_search` 函数）

&emsp;&emsp;在`run_local_search` 的工作流的第一个阶段，会依次读取`text_units.parent`、`entities.parent`、`relationships.parent` `communities.parent` 和`community_reports.parent` 的索引文件，并将其加载到内存中。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141121873.png" width=80%></div>

&emsp;&emsp;第二个阶段，在构建索引的最后一个`workflow` 中，会执行`get_embedding`，即将各个`.parent` 文件中实体、关系、内容、社区摘要、社区报告等等数据转换为词向量，并存储至本地的`lancedb` 离线索引中。这检索时，会加载`Lancedb` 中的词向量，准备用于后续的相似度计算。不过需要注意的是：因为`Local Search` 是基于实体的检索，所以在这个阶段，只会将`entities.parent` 文件中的数据转换为词向量。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141150541.png" width=80%></div>

&emsp;&emsp;第三个阶段，会根据社区的`community_level` 参数的值，对实体进行第一轮的过滤，过滤的规则是：如果实体的`community_level` 小于等于`community_level` 参数的值，则保留该实体，否则丢弃。即可以通过`poetry run poe query --method local --community-level 3 --query "苹果公司都有哪些产品？"` 来指定检索的社区级别。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141206355.png" width=80%></div>

&emsp;&emsp;第四个阶段，基于输入的问题，进行实体的匹配，并构建完整的上下文。其中详细的细节如下：

1. 处理输入问题，如果存在对话历史记录，则将之前的用户问题附加到当前查询。
2. 将输入的问题转化为词向量，然后和lancedb中的实体词向量进行相似度计算，得到与查询最相关的实体，这个过程中会采用两个策略：
    - 过采样 (Oversampling) 策略，即最终检索的实体数量是 k * oversample_scaler。
    - 如果提供了exclude_entity_names列表，则过滤掉这些实体。
3. 根据匹配到的实体，读取该实体所属的社区报告，这个过程会采用的策略是：
    - 统计每个社区被多少个选中实体引用（一个实体可能属于多个社区），做基于实体归属的社区投票排序
    - 按匹配度和社区自身排名双重排序
        - 主要排序标准：被实体引用的频次（匹配度）
        - 次要排序标准：社区自身的重要性排名
4. 在 2,3 的基础上，提取出文本单元、关系的附件属性
5. 生成完整的数据表格

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141355052.png" width=80%></div>

&emsp;&emsp;最后的第五阶段，构建本地搜索的系统提示词，将数据表格填充到系统提示词中，引导大模型生成最终的回答。其提示词设置在`settings.yaml` 文件的`local_search` 中，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141406170.png" width=80%></div>

&emsp;&emsp;以上就是`Local Search` 的完整实现过程，至此，大家再理解`Microsoft GraphRAG` 给出的`Local Search` 原理图，就可以非常清晰的理解到其中各个阶段的工作原理了。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141408842.png" width=80%></div>

&emsp;&emsp;好，`Local Search` 的源码解析到这里就全部结束了。所以的细节均给大家介绍完毕。接下来我们在关注`Global Search` 的实现过程。

# 2. Global Search 实现原理与源码解析

&emsp;&emsp;`Microsoft GraphRAG` 中的全局搜索算法旨在回答需要了解整个数据集的抽象问题，即借助社区摘要来获取全局的答案。实现思路是通过 map-reduce 流程总结知识图谱中的社群摘要，汇总社区摘要中的见解，尝试生成文档中元素的概述，聚合相关资料并生成针对整体数据集的高层次回答。因此全局搜索更侧重于为需要更高层次理解的问题提供答案。比如`数据中的前5个主题是什么？`这类问题。


&emsp;&emsp;传统的 `Map-Reduce` 中，`Map`阶段会将输入数据分解为独立的块，每个块可以并行处理，然后`Reduce` 阶段会汇总所有映射操作的结果，生成最终输出。而在`Microsoft GraphRAG` 中，`Map` 阶段会使用大模型对多个文档或信息片段并行处理，从每个片段中提取相关信息，然后`Reduce` 阶段会汇总所有映射操作的结果，生成最终输出。

&emsp;&emsp;`Global Search` 在`Microsoft GraphRAG` 源码中的实现原理图如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141417825.png" width=80%></div>

&emsp;&emsp;与`Local Search` 类似，`Global Search` 的实现过程也是分阶段进行的。其中主入口文件，仍然在`graphrag/cli/main.py` 文件中，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141418777.png" width=80%></div>

&emsp;&emsp;如果大家完整的实现了`Local Search` 的源码解析，那么`Global Search` 的源码解析就非常简单了。按照相同的思路，逐步进入到各个阶段的内部细节即可清晰的了解其实现的原理。接下来我们就分步骤给大家介绍`Global Search` 的实现过程。


&emsp;&emsp;注意，当使用`Global Search` 时，需要指定`--method global` 参数，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141644641.png" width=80%></div>

&emsp;&emsp;在`run_global_search` 的工作流的第一个阶段，会依次读取`entities.parent`、`communities.parent` 和`community_reports.parent` 的索引文件，并将其加载到内存中。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141422011.png" width=80%></div>

&emsp;&emsp;在第二个阶段，依次创建 `entities.parent`、`communities.parent` 和`community_reports.parent` 的实体对象，并进行格式化处理。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141455523.png" width=80%></div>

&emsp;&emsp;第三个阶段，会进入到构建上下文阶段。在这个阶段，最关键的一个核心概念是：静态与动态全局搜索策略的选择。

&emsp;&emsp;静态策略方法指的是知识图谱中预定级别的社区中进行搜索来生成答案。然后，大模型合并并总结此抽象级别的所有社区报告。最后，摘要用作 大模型的附加上下文，以生成对用户问题的响应。此为静态方法。它存在的问题是既昂贵又低效，因为包含许多对用户查询没有帮助的低级报告。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141521048.png" width=80%></div>

&emsp;&emsp;而`Microsoft GraphRAG` 中的全局搜索算法的主要做用是回答需要了解整个数据集的抽象问题。因此引入了动态社区选择算法`dynamic_community_selection`。该算法利用索引数据集的知识图谱结构。从知识图谱的根开始，我们使用`提示工程 + 大模型`来评估社区报告在回答用户问题方面的相关性。如果报告被视为不相关，则将其及其节点（或子社区）从搜索过程中删除。另一方面，如果报告被视为相关，将遍历其子节点并重复该操作。最后，只有相关的报告才会传递给 `map-reduce` 操作以生成对用户的响应。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141445707.png" width=80%></div>

&emsp;&emsp;这个核心的算法结构其定义的源码位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141527021.png" width=80%></div>

&emsp;&emsp;该算法类实现的核心机制并不是简单地评估所有社区，而是采用启发式遍历，具体体现在：

1. **选择性探索**: 并不是简单地评估所有社区，而是根据当前社区的相关性决定是否探索其子社区。只有当一个社区的评分大于或等于阈值时，才会将其子社区添加到下一轮评估中。
2. **动态队列构建**: queue = communities_to_rate 表明下一轮要评估的社区完全取决于当前轮次中哪些社区被认为是相关的。这不是一个固定的或预先确定的遍历顺序。
3. **剪枝机制**: 如果一个社区的评分低于阈值，其所有子社区都会被"剪枝"，不会被进一步探索。这是启发式算法的典型特征。
4. **自适应性**: 算法的路径会根据不同的查询而变化，因为相关性评分依赖于具体的查询内容。
5. **回退策略**: 如果在当前路径上找不到相关社区，算法会尝试探索下一个层级的所有社区，这也是一种启发式决策。

&emsp;&emsp;其中，用于评估社区相关性的提示词定义在`graphrag/query/context_builder/rate_prompt.py` 文件中，其对应的中文提示如下所示：

```json
    Rate_query = "“”
    ——角色
    你是一个乐于助人的助手，负责决定所提供的信息是否有助于回答给定的问题，即使它只是部分相关。
    ——目标
    在0到5的范围内，请对回答问题所提供的信息的相关性或帮助程度进行评分。
    ——信息
    {描述}
    ——问题
    {问题}
    ——目标回复长度和格式——
    请以以下JSON格式回复，包含两个条目：
    -“原因”：评分的原因，请包括你考虑过的信息。
    -“评级”：相关度从0到5，其中0是最不相关的，5是最相关的。
    {{
    “理由”:str,
    “等级”:int。
    }}
```


&emsp;&emsp;在`rate_relevancy` 函数中，会使用大模型来评估查询与社区描述之间的相关性，返回0-5的评分值。最后，当构建完整的社区上下文时，依次调用`prompts/global_search_map_system_prompt.txt`，将完整的上下文数据填充到提示词中，引导大模型生成最终的Map 响应。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141603231.png" width=80%></div>

&emsp;&emsp;其响应的结果如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141605641.png" width=80%></div>

&emsp;&emsp;然后，该输出的结果会作为变量`context_data` 传递给`global_search_reduce_system_prompt.txt` 中定义的提示，并调用大模型生成最终的 `Reduce` 响应，同时，`Reduce` 响应的结果会作为变量`context_text` 传递给`global_search_knowledge_system_prompt.txt` 中定义的提示，引导大模型生成最终的 `Knowledge` 响应。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141623652.png" width=80%></div>

&emsp;&emsp;至此，`Global Search` 的实现原理就全部介绍完毕了。而现在大家在看`Microsoft GraphRAG` 官方提供的流程图，自然也能够非常清晰的理解其背后的实现原理。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503141625872.png" width=80%></div>

&emsp;&emsp;以上就是我们对`Microsoft GraphRAG` 的检索流程中最关键的`Local Search` 和 `Global Search`的完整流程解读，而关于`basic search`和`drift search`，大家可以按照相同的思路进行扩展学习，我们也将在接下来的实战课程中，进一步给大家展开详细的介绍。